In [1]:
!pip install tensorflow_text

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np

2022-09-15 03:51:08.980640: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataFrame= pd.read_json('/Users/Bobbie/Downloads/amazon_review/reviews_Automotive_5.json',lines=True)

In [4]:
dataFrame.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"
5,A1ORODEBRN64C,B00002243X,"James F. Magowan ""Jimmy Mac""","[1, 1]","These Jumper cables are heavy Duty, Yet easy t...",5,Compact and Strong !,1258156800,"11 14, 2009"
6,A2R49ZN3G6FTCQ,B00002243X,John M. Harrell,"[1, 1]",bought these for my k2500 suburban plenty of l...,5,nice cables,1326153600,"01 10, 2012"
7,A1Q65KYDKXIX8E,B00002243X,Leeland H.,"[0, 0]",these are good enough to get most motorized ve...,5,for cars and pickups,1371081600,"06 13, 2013"
8,A3BI8BKIHESDNQ,B00002243X,L. J. Cunningham,"[0, 0]",The Coleman Cable 08665 12-Feet Heavy-Duty Tru...,4,Coleman Cable 08665 12-Feet Heavy-Duty Truck a...,1374105600,"07 18, 2013"
9,A1R089P5AS26UE,B00002243X,Mike,"[0, 0]","I have an old car, Its bound to need these som...",5,Beefy,1390348800,"01 22, 2014"


In [5]:
dataFrame['overall'].value_counts()

5    13928
4     3967
3     1430
2      606
1      542
Name: overall, dtype: int64

In [6]:
dataFrame['Identifier']=dataFrame['overall'].apply(lambda x:1 if x>=4 else 0)

In [7]:
dataFrame.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Identifier
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011",1
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4,Okay long cables,1315094400,"09 4, 2011",1
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5,Looks and feels heavy Duty,1374710400,"07 25, 2013",1
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010",1
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012",1
5,A1ORODEBRN64C,B00002243X,"James F. Magowan ""Jimmy Mac""","[1, 1]","These Jumper cables are heavy Duty, Yet easy t...",5,Compact and Strong !,1258156800,"11 14, 2009",1
6,A2R49ZN3G6FTCQ,B00002243X,John M. Harrell,"[1, 1]",bought these for my k2500 suburban plenty of l...,5,nice cables,1326153600,"01 10, 2012",1
7,A1Q65KYDKXIX8E,B00002243X,Leeland H.,"[0, 0]",these are good enough to get most motorized ve...,5,for cars and pickups,1371081600,"06 13, 2013",1
8,A3BI8BKIHESDNQ,B00002243X,L. J. Cunningham,"[0, 0]",The Coleman Cable 08665 12-Feet Heavy-Duty Tru...,4,Coleman Cable 08665 12-Feet Heavy-Duty Truck a...,1374105600,"07 18, 2013",1
9,A1R089P5AS26UE,B00002243X,Mike,"[0, 0]","I have an old car, Its bound to need these som...",5,Beefy,1390348800,"01 22, 2014",1


In [8]:
dataFrame['Identifier'].value_counts()

1    17895
0     2578
Name: Identifier, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split 

In [10]:
x_train,x_test,y_train,y_test=train_test_split(dataFrame['reviewText'],dataFrame['Identifier'])

In [11]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

model_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [12]:
bert_preprocess_model= hub.KerasLayer(preprocess_url)

2022-09-15 03:52:17.942686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
bert_model_url=hub.KerasLayer(model_url)

In [14]:
myinputs=tf.keras.layers.Input(shape=(),dtype=tf.string,name='InputLayer')

preprocessed_text= bert_preprocess_model(myinputs)

myoutputs=bert_model_url(preprocessed_text)['pooled_output']

myoutputs= tf.keras.layers.Dense(64,activation='relu',name='HiddenLayer')(myoutputs)

myoutputs=tf.keras.layers.Dense(1,activation='sigmoid',name='Outputyer')(myoutputs)

model = tf.keras.Model(inputs=myinputs, outputs=myoutputs)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLayer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['InputLayer[0][0]']             
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [16]:
model.compile(loss='binary_crossentropy'
                     ,optimizer='adam'
                     ,metrics=['accuracy'])

In [17]:
model.fit(x_train,y_train,epochs=1)

480/480 [==============================] - 2473s 5s/step - loss: 0.3682 - accuracy: 0.8725


In [19]:
model.save('/Users/Bobbie/Downloads/amazon_review/1')

INFO:tensorflow:Assets written to: /Users/Bobbie/Downloads/amazon_review/1/assets


INFO:tensorflow:Assets written to: /Users/Bobbie/Downloads/amazon_review/1/assets
